---
---
# **<center>PROYECTO FINAL: GOOGLE & YELP<center>**

---

---

### Mediante el siguiente Notebook se realiza un análisis exploratorio(EDA) de los datasets correspondientes a las reviews obtenidas de la plataforma google maps, se trabajarán los datos de unicamente 4 estados , Pensilvania, Florida, California e Ilinois, los mismos fueron seleccionados en base a la cantidad de negocios por estados.

In [ ]:
# Importar librerias a utilizar
from google.colab import drive
import pandas as pd
import numpy as np
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import gdown
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import seaborn as sns



In [ ]:
!pip install nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Establecer conexión con google Drive
drive.mount('/content/drive')

Mounted at /content/drive


##*Lectura de datos - "Google maps-reviews-estados"*
---


In [ ]:
# Lee los datos de "user_reviews" extraidos de Google Maps y retorna un dataframe de pandas con los datos
def download_and_read(file_id):
    url = f'https://drive.google.com/uc?id={file_id}'
    output = 'google.json'
    gdown.download(url, output, quiet=False)
    return pd.read_json(output, lines=True)

# Reemplaza 'file_id' con el ID de tu archivo en Google Drive
PA = download_and_read('1JOP4_XxxRGPsqTl0tZHNXYQg221qM24p')
FL = download_and_read('1cFw3IyigV53ngxfBy4U9VJfu8yILSHcv')
CA = download_and_read('13JlGdagtTp4SrUIXu5osayX0f-vmeMz6')
IL = download_and_read('1ZOJjSC6cudmNj8W3PBuKLvQEr18R7akt')

Downloading...
From: https://drive.google.com/uc?id=1JOP4_XxxRGPsqTl0tZHNXYQg221qM24p
To: /content/google.json
100%|██████████| 45.1M/45.1M [00:00<00:00, 105MB/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
Downloading...
From: https://drive.google.com/uc?id=1cFw3IyigV53ngxfBy4U9VJfu8yILSHcv
To: /content/google.json
100%|██████████| 56.1M/56.1M [00:00<00:00, 107MB/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
Downloading...
From: https://drive.google.com/uc?id=13JlGdagtTp4SrUIXu5osayX0f-vmeMz6
To: /content/google.json
100%|██████████| 47.9M/47.9M [00:00<00:00, 164MB/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
Downloading...
F

In [ ]:
# Unir los DataFrames de cada estado
df_PAFLCAIL = pd.concat([PA, FL, CA, IL], ignore_index=True)

In [ ]:
df_PAFLCAIL.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.049447e+20,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb
1,1.176097e+20,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb
2,1.105632e+20,Rocky Kev,1338826945578,5,I usually give them a call before I stop by to...,None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb
3,1.128960e+20,Josep Valls,1363286110554,5,My bike had been sitting outdoors for a good w...,None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb
4,1.106197e+20,Timaree Schmit,1548798329760,5,Always an easy experience. Service is knowledg...,None,None,0x89c6c63c8cd87141:0x54d0d283872eecbb


### A continuación se cruzarán los datos de "user_reviews" con los datos de "Metadata_sitios" y se realizará un analisis exploratorio de los datos resultantes.

In [ ]:
# Lee los datos del dataset "metadata_sitios" y devuelve un dataframe de pandas
def download_and_read(file_id):
    url = f'https://drive.google.com/uc?id={file_id}'
    output = 'googlemetadata.json'
    gdown.download(url, output, quiet=False)
    return pd.read_json(output, lines=True)

# Reemplaza 'file_id' con el ID de tu archivo en Google Drive
metadata = download_and_read('1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS')

Downloading...
From (original): https://drive.google.com/uc?id=1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS
From (redirected): https://drive.google.com/uc?id=1OnyhmyG8xzdn4XU9LYcnwzBseB1_rChS&confirm=t&uuid=f2622b1c-0d9b-4f32-9515-26c565b207d0
To: /content/googlemetadata.json
100%|██████████| 256M/256M [00:01<00:00, 148MB/s]


In [ ]:
df_PAFLCAIL_final = df_PAFLCAIL[['text', 'gmap_id']]

In [ ]:
metadata.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')

In [ ]:
metadata.head()


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [ ]:
# Selecciona las columnas 'gmap_id' y 'category' de metadata
metadata_selected = metadata[['gmap_id', 'category']]

# Selecciona las columnas 'gmap_id' y 'text' de df_PAFLCAIL
df_PAFLCAIL_selected = df_PAFLCAIL[['gmap_id', 'text']]

# Une los dos DataFrames en base a 'gmap_id'
df_merged = pd.merge(metadata_selected, df_PAFLCAIL_selected, on='gmap_id')

In [ ]:
df_merged.head()

,gmap_id,category,text
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant],Love there korean rice cake.
1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant],Good very good
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant],They make Korean traditional food very properly.
3,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant],Short ribs are very delicious.
4,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant],Great food and prices the portions are large


In [ ]:
# Veamos la info completa
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348752 entries, 0 to 348751
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   gmap_id   348752 non-null  object
 1   category  348327 non-null  object
 2   text      216742 non-null  object
dtypes: object(3)
memory usage: 10.6+ MB


In [ ]:
# 'explode' convierte listas en filas separadas
df_exploded = df_merged.explode('category')

# Ahora puedes contar los valores únicos
unique_values = df_exploded['category'].nunique()

print(f"Valores únicos en 'category': {unique_values}")

Valores únicos en 'category': 2289


In [ ]:
total_values_per_column = df_merged.count()
print(total_values_per_column)

gmap_id     348752
category    348327
text        216742
dtype: int64


In [ ]:
palabras_clave = ['Burger', 'BBQ', 'Tex-Mex', 'Soul Food', 'Cajun', 'Creole', 'Hot Dog', 'Buffalo Wings', 'Clam Chowder', 'Gumbo', 'Jambalaya', 'Hushpuppies', 'Cheesesteak', 'Philly Pretzel', 'Lobster Roll', 'New England Clam Bake', 'Chicago Deep Dish Pizza', 'Thin Crust Pizza', 'Bagel', 'Reuben Sandwich', 'Po boy', 'Tater Tots', 'Cornbread', 'Fried Chicken', 'Chicken and Waffles', 'Biscuits and Gravy', 'Grits', 'Shrimp and Grits', 'Collard Greens', 'Pulled Pork', 'Ribs', 'Brisket', 'Sloppy Joe', 'Chili', 'Buffalo Chicken', 'Fried Green Tomatoes', 'Pecan Pie', 'Key Lime Pie', 'Pumpkin Pie', 'Apple Pie', 'Peach Cobbler', 'Banana Pudding', 'Red Velvet Cake', 'Gumbo', 'Shrimp Po Boy', 'Beignets', 'Tacos', 'Burritos', 'Enchiladas', 'Nachos', 'Quesadillas', 'Guacamole', 'Salsa', 'Margarita', 'Mojito', 'Mint Julep', 'Craft Beer', 'Microbrewery', 'Distillery', 'Bourbon', 'Whiskey', 'Moonshine', 'Sweet Tea', 'Iced Coffee', 'Pumpkin Spice Latte', 'Corn Dog', 'State Fair Food', 'Food Truck', 'Food Festival', 'Street Food', 'Farmers Market', 'Apple Cider', 'Clam Bake', 'Lobster Bake', 'Lobster Mac and Cheese', 'Lobster Bisque', 'Crab Cakes', 'Maryland Crab Soup', 'Chesapeake Bay Oysters', 'Manhattan Clam Chowder', 'Lobster Newberg', 'Manhattan', 'Martini', 'Craft Cocktail', 'Moon Pie', 'Fried Oreos', 'Pimento Cheese', 'Shrimp Cocktail', 'Oyster Rockefeller', 'Oyster PoBoy', 'Smoked Salmon Bagel', 'Bagel and Lox', 'Pretzel Bun', 'Buffalo Chicken Dip', 'Tuna Melt', 'Caesar Salad', 'Cobb Salad', 'Waldorf Salad', 'Monte Cristo Sandwich', 'King Cake', 'Grits', 'Pimento Cheese Grits', 'Chicken Fried Steak', 'Chicken Pot Pie', 'California Roll', 'Philly Roll', 'Sushi', 'Ramen', 'Dim Sum', 'General Tso s Chicken', 'Peking Duck', 'Egg Drop Soup', 'Chop Suey', 'Moo Shu Pork', 'California Burrito', 'Fish Tacos', 'Avocado Toast', 'Quinoa Bowl', 'Acai Bowl', 'Poke Bowl', 'Kale Salad', 'Green Goddess Dressing', 'Artisanal Cheese', 'Charcuterie', 'Truffle Fries', 'Lobster Risotto', 'Shrimp and Grits', 'Bourbon Bacon Jam', 'Sriracha', 'Sriracha Mayo', 'Ranch Dressing', 'Blue Cheese Dressing', 'Thousand Island Dressing', 'Barbecue Sauce', 'Hot Sauce', 'Mustard', 'Ketchup', 'Mayonnaise', 'Poutine', 'Maple Syrup', 'Clamato', 'Vodka Infusions', 'Avocado Ice Cream', 'Maple Bacon Ice Cream', 'Deep-Fried Ice Cream', 'Beignet Sundae', 'Biscuit Donuts', 'Fried Pickles', 'Buffalo Cauliflower', 'Gouda Mac and Cheese', 'Pulled Pork Nachos', 'Bratwurst', 'Bison Burger', 'Elk Burger', 'Duck Confit', 'Chicken and Dumplings', 'Buttermilk Pie', 'Red Beans and Rice', 'Collard Greens with Ham Hock', 'Bourbon Pecan Pie', 'Bourbon Balls', 'Moonshine Jelly', 'Bourbon Bacon Jam', 'Gator Tail', 'Crawfish Boil', 'Andouille Sausage', 'Fried Green Tomato BLT', 'Catfish Po Boy', 'Shrimp Gumbo', 'Alligator Bites', 'Grilled Cheese with Tomato Soup', 'Chicken Parmesan', 'Chicken Marsala', 'Chicken Alfredo', 'Caesar Wrap', 'Club Sandwich', 'Caprese Salad', 'Clam Linguine', 'Crab Rangoon', 'Crawfish Etouffee', 'Cuban Sandwich', 'Dutch Baby Pancake', 'Elk Chili', 'Fish and Chips', 'French Dip', 'Fried Catfish', 'Frog Legs', 'Gator PoBoy', 'General Tsos Tofu', 'Grits and Grillades', 'Hoppin John', 'Hot Chicken', 'King Ranch Casserole', 'Livermush', 'Matzo Ball Soup', 'Monte Cristo Sandwich', 'Muffuletta', 'Nashville Hot Chicken', 'Navajo Taco', 'Okra Fritters', 'Pecan Pralines', 'Pickled Okra', 'Pig Pickin', 'Rattlesnake Chili', 'Rocky Mountain Oysters', 'Shrimp and Okra Gumbo', 'Smothered Pork Chops', 'Sonoran Hot Dog', 'Sourdough Bread Bowl', 'Southern Fried Catfish', 'Spam Musubi', 'Squash Casserole', 'Tamales', 'Turducken', 'White BBQ Sauce', 'White Castle Slider', 'Zucchini Bread']  # Asegúrate de reemplazar '...' con tus palabras clave

# Crea una expresión regular a partir de las palabras clave
patron = '|'.join(palabras_clave)

# Crea una nueva columna que indica si la categoría contiene alguna de las palabras clave
df_merged['es_restaurante'] = df_merged['category'].apply(lambda x: bool(re.search(patron, str(x), re.IGNORECASE)) if x is not None else False)

# Filtra solo los restaurantes
df_restaurantes_merged = df_merged[df_merged['es_restaurante']]

# Repite el proceso para df_exploded si es necesario
df_exploded['es_restaurante'] = df_exploded['category'].apply(lambda x: bool(re.search(patron, str(x), re.IGNORECASE)) if x is not None else False)
df_restaurantes_exploded = df_exploded[df_exploded['es_restaurante']]

In [ ]:
# 'Explota' la columna 'category' para que cada elemento de las listas tenga su propia fila
df_exploded = df_restaurantes_merged.explode('category')

# Ahora puedes contar los valores únicos
num_categorias_unicas = df_exploded['category'].nunique()
print(f"La cantidad de valores únicos en la columna 'category' es: {num_categorias_unicas}")

La cantidad de valores únicos en la columna 'category' es: 103


In [ ]:
# Ahora puedes contar el total de valores
num_categorias = df_exploded['category'].count()
print(f"La cantidad total de valores en la columna 'category' es: {num_categorias}")

La cantidad total de valores en la columna 'category' es: 10228


In [ ]:
# Ahora puedes contar el total de valores
num_categorias = df_exploded['es_restaurante'].count()
print(f"La cantidad total de valores en la columna 'category' es: {num_categorias}")

La cantidad total de valores en la columna 'category' es: 10228


In [ ]:
# Obtiene una lista de los valores en la columna 'category'
lista_categorias = df_exploded['category'].tolist()

# Imprime la lista de categorías
print(lista_categorias)

['Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant', 'Fast food restaurant', 'Mexican restaurant', 'Sandwich shop', 'Hamburger restaurant', 'American restaurant', 'Barbecue restaurant'

In [ ]:
# Obtiene una serie con los recuentos de valores únicos
recuentos = df_exploded['category'].value_counts()

# Obtiene las 10 palabras más repetidas
top_10 = recuentos.head(10)

# Imprime el top 10 de palabras más repetidas
print(top_10)

Fast food restaurant    847
Hamburger restaurant    663
Restaurant              660
Sushi restaurant        598
Breakfast restaurant    442
Sandwich shop           428
American restaurant     362
Coffee shop             348
Japanese restaurant     287
Takeout Restaurant      280
Name: category, dtype: int64


In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Crea un analizador de sentimiento
sia = SentimentIntensityAnalyzer()


In [ ]:
# Aplica el analizador de sentimiento a cada texto
df_restaurantes_merged['sentiment_score'] = df_restaurantes_merged['text'].apply(lambda text: sia.polarity_scores(text)['compound'] if text is not None else 0)

# Ajusta las puntuaciones de sentimiento para que se ajusten a la escala de -2 a 2
df_restaurantes_merged['sentiment_score_scaled'] = df_restaurantes_merged['sentiment_score'].apply(lambda score: np.interp(score, [-1, 1], [-2, 2]))

# Asigna cada rango a una etiqueta de sentimiento
bins = [-np.inf, -1.2, -0.2, 0.2, 1.2, np.inf]
labels = ['pesimo', 'malo', 'neutral', 'bueno', 'excelente']
df_restaurantes_merged['sentiment_label'] = pd.cut(df_restaurantes_merged['sentiment_score_scaled'], bins=bins, labels=labels)

# Ahora, la columna 'sentiment_label' contiene la etiqueta de sentimiento para cada texto


<ipython-input-24-4bf72ffdd3c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_restaurantes_merged['sentiment_score'] = df_restaurantes_merged['text'].apply(lambda text: sia.polarity_scores(text)['compound'] if text is not None else 0)
<ipython-input-24-4bf72ffdd3c3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_restaurantes_merged['sentiment_score_scaled'] = df_restaurantes_merged['sentiment_score'].apply(lambda score: np.interp(score, [-1, 1], [-2, 2]))
<ipython-input-24-4bf72ffdd3c3>:10: Setti

In [ ]:
# Haz una copia profunda del DataFrame
df = df_restaurantes_merged.copy(deep=True)

# Aplica el analizador de sentimiento a cada texto
df['sentiment_score'] = df['text'].apply(lambda text: sia.polarity_scores(text)['compound'] if text is not None else 0)

# Ajusta las puntuaciones de sentimiento para que se ajusten a la escala de -2 a 2
df['sentiment_score_scaled'] = df['sentiment_score'].apply(lambda score: np.interp(score, [-1, 1], [-2, 2]))

# Asigna cada rango a una etiqueta de sentimiento
bins = [-np.inf, -1.2, -0.2, 0.2, 1.2, np.inf]
labels = ['pesimo', 'malo', 'neutral', 'bueno', 'excelente']
df['sentiment_label'] = pd.cut(df['sentiment_score_scaled'], bins=bins, labels=labels)

# Ahora, la columna 'sentiment_label' contiene la etiqueta de sentimiento para cada texto


In [ ]:
# Obtiene una serie con los recuentos de cada categoría de sentimiento
recuentos_sentimientos = df_restaurantes_merged['sentiment_label'].value_counts()

# Imprime los recuentos de cada categoría de sentimiento
print(recuentos_sentimientos)


neutral      972
excelente    885
bueno        308
malo         101
pesimo        82
Name: sentiment_label, dtype: int64
